In [14]:
import itertools
import json
from pathlib import Path

import pandas as pd

In [15]:
def product_experiment_configs(params_record):
    # Generate all possible combinations of parameters
    keys = params_record.keys()
    values = params_record.values()
    for instance in itertools.product(*values):
        yield dict(zip(keys, instance))

In [16]:
common_params = {
    "params.dataset.path": ["bdsaglam/musique-sweep"],
    "params.dataset.split": ["train"],
    "params.qa.temperature": [
        # 0.1,
        0.5,
        # 0.7,
        # 1.0,
        # 1.5,
    ],
    "params.qa.user_prompt_template": ["cq.txt"],
    "params.qa.n_shot": [
        0,
        1,
        2,
        # 3,
        # 6,
        # 9,
    ],
    "params.run": [
        1,
        # 2,
        # 3,
    ],
}
params_records = [
    # {
    #     "params.qa.system_prompt": [
    #         "direct/format-few.txt",
    #         "direct/excellent-format-few.txt",
    #     ],
    #     "params.qa.few_shot_examples": ["manual/direct.json"],
    # },
    {
        "params.qa.system_prompt": [
            "cot/format-thought.txt",
            # "cot/excellent-format-thought.txt",
        ],
        "params.qa.few_shot_examples": ["manual/cot.json"],
    },
    # {
    #     "params.qa.system_prompt": [
    #         "ccot/format-thought.txt",
    #         "ccot/excellent-format-thought.txt",
    #     ],
    #     "params.qa.few_shot_examples": ["manual/ccot.json"],
    # },
    # {
    #     "params.qa.system_prompt": [
    #         "cte/format-triplets-few.txt",
    #         "cte/excellent-format-triplets-few.txt",
    #     ],
    #     "params.qa.few_shot_examples": ["manual/cte.json"],
    # },
]

exp_configs = [
    exp_config
    for params_record in params_records
    for exp_config in product_experiment_configs({**common_params, **params_record})
]
print(f"{len(exp_configs)} experiment configurations")

3 experiment configurations


In [17]:
target_params = exp_configs[0].keys()
target_params

dict_keys(['params.dataset.path', 'params.dataset.split', 'params.qa.temperature', 'params.qa.user_prompt_template', 'params.qa.n_shot', 'params.run', 'params.qa.system_prompt', 'params.qa.few_shot_examples'])

In [18]:
results_path = Path("exps.jsonl")
if results_path.exists():
    with open(results_path) as f:
        experiments = [json.loads(line) for line in f]
else:
    experiments = []

print(f"{len(experiments)} experiments")
next(iter(experiments), None)

1 experiments


{'id': '402ac0f6ae24febd83b5d3889c958f02114ce7a5',
 'name': 'oiled-lulu',
 'params.dataset.path': 'bdsaglam/musique-sweep',
 'params.dataset.name': 'answerable',
 'params.dataset.split': 'train',
 'params.qa.model': 'llama-3-70b-tgi',
 'params.qa.temperature': 1.5,
 'params.qa.system_prompt': 'cot/excellent-format-thought.txt',
 'params.qa.user_prompt_template': 'cq.txt',
 'params.qa.few_shot_examples': 'auto-interweave/cot.json',
 'params.qa.n_shot': 9,
 'params.run': 3,
 'metrics.exact_match': 0.0033333333,
 'metrics.f1': 0.0047530612,
 'metrics.2hops.exact_match': 0.01,
 'metrics.2hops.f1': 0.0100122699,
 'metrics.3hops.exact_match': 0.0,
 'metrics.3hops.f1': 0.0,
 'metrics.4hops.exact_match': 0.0,
 'metrics.4hops.f1': 0.0042469136,
 'metrics.gen_token_count.all.count': 300.0,
 'metrics.gen_token_count.all.mean': 182.0866666667,
 'metrics.gen_token_count.all.std': 493.3772419069,
 'metrics.gen_token_count.all.min': 0.0,
 'metrics.gen_token_count.all.25%': 7.0,
 'metrics.gen_token_co

In [19]:
df = pd.DataFrame(experiments)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]

df.dropna(subset=param_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

1 experiments after preprocessing


In [20]:
existing_configs = df[target_params].to_dict(orient="records")
existing_configs[0]

{'params.dataset.path': 'bdsaglam/musique-sweep',
 'params.dataset.split': 'train',
 'params.qa.temperature': 1.5,
 'params.qa.user_prompt_template': 'cq.txt',
 'params.qa.n_shot': 9,
 'params.run': 3,
 'params.qa.system_prompt': 'cot/excellent-format-thought.txt',
 'params.qa.few_shot_examples': 'auto-interweave/cot.json'}

In [21]:
target_params = list({**common_params, **params_records[0]}.keys())
target_params.remove("params.run")
target_params

['params.dataset.path',
 'params.dataset.split',
 'params.qa.temperature',
 'params.qa.user_prompt_template',
 'params.qa.n_shot',
 'params.qa.system_prompt',
 'params.qa.few_shot_examples']

In [23]:
# find the missing configurations
missing_configs = [
    dict(kv)
    for kv in list(
        {tuple(sorted(config.items())) for config in exp_configs}
        - {tuple(sorted(config.items())) for config in existing_configs}
    )
]
print(f"{len(missing_configs)} missing configurations")
next(iter(missing_configs), None)


3 missing configurations


{'params.dataset.path': 'bdsaglam/musique-sweep',
 'params.dataset.split': 'train',
 'params.qa.few_shot_examples': 'manual/cot.json',
 'params.qa.n_shot': 0,
 'params.qa.system_prompt': 'cot/format-thought.txt',
 'params.qa.temperature': 0.5,
 'params.qa.user_prompt_template': 'cq.txt',
 'params.run': 1}

In [24]:
def make_command(exp_config):
    run = exp_config["params.run"]
    lines = ["dvc exp run --queue", f"-S run={run}"]
    for target_param in target_params:
        arg_name = target_param.split(".", 1)[-1]
        arg_value = exp_config[target_param]
        lines.append(f"-S {arg_name}='{arg_value}'")

    command = " \\\n    ".join(lines)
    return command

In [25]:
with open("run.sh", "w") as f:
    f.write("#!/bin/sh\n")
    for exp_config in missing_configs:
        f.write(make_command(exp_config))
        f.write("\n\n")